In [ ]:
import allel
import pandas as pd
import numpy as np
import plotly.express as px

In [ ]:
metadata_path = '../../../config/metadata.tsv'
bed_targets_path = "../../../config/AgamDao.bed"
vcf_path = "vcf"
wkdir = "../.."

In [ ]:
metadata = pd.read_csv(metadata_path , sep="\t")
targets = pd.read_csv(bed_targets_path, sep="\t", header=None)
targets.columns = ['contig', 'start', 'end', 'amplicon', 'mutation']

vcf = allel.read_vcf(vcf_path)

samples = vcf['samples']
contigs = vcf['variants/CHROM']
geno = allel.GenotypeArray(vcf['calldata/GT'])
pos = vcf['variants/POS']
ref = vcf['variants/REF']

In [ ]:
aim_targets = targets.query("mutation.str.contains('AIM')", engine='python')

aim_mask = np.isin(pos, aim_targets.end.to_list())
aim_gn = geno.compress(aim_mask, axis=0)
aim_pos = pos[aim_mask]
aim_contig = contigs[aim_mask]

aim_df = pd.DataFrame(aim_gn.to_n_alt(fill=-1)).assign(pos=aim_pos, contig=aim_contig).set_index(['pos', 'contig'])
# reorder contigs 
aim_df = pd.concat([aim_df.query("contig == @contig") for contig in ['2R', '2L', '3R', '3L', 'X']])
aim_pos = aim_df.reset_index()['pos']
aim_contig = aim_df.reset_index()['contig']

x_label = [f"{c}:{p}" for c, p in zip(aim_contig, aim_pos)]

fig = px.imshow(
    aim_df.values.T, 
    x=x_label,
    y=samples, 
    color_continuous_scale='blackbody_r',
    aspect='auto',
    title='gamb vs colu AIM genotypes',
    width=1000, 
    height=800
)
fig.show()